# Object detection
---

**Object detection**이란 이미지 내에서 물체의 위치와 그 종류를 찾아내는 것이다.


이번 학습을 통해 `딥러닝 기반의 Object detection 기법`, `Anchor box의 개념`, `single stage detection model과 two stage detection 모델의 차이`에 대해 알아보자



## 관련 용어 (object Localization, bounding box, intersection over union)


### 1) Object Localization
---

![image](https://user-images.githubusercontent.com/70703320/114338859-e683f100-9b8e-11eb-84c8-ba7f338dd489.png)

[출처:https://medium.com/@prashant.brahmbhatt32/the-yolo-object-detection-c195994b53aa]

`Object detection`은 물체를 `분류(classification)` + `위치 측정 (localization)`을 함께 수행하는 작업이다.

`Localization`이란 이미지 내에 **하나의 물체(Object)** 가 있을 때 그 **물체의 위치를 특정**하는 것이다.

따라서, `Detection`이라 함은 **다수의 물체(Object)** 가 존재할 때 각 Object의 **존재 여부를 파악**하고 **위치를 특정**하며 **클래스 분류(Classification)** 까지 수행하는 것이다.

object detection을 나타내는 방법 중 하나가 bounding box이다.


### 2) 바운딩 박스(Bounding Box)
---

![image](https://user-images.githubusercontent.com/70703320/114339710-cead6c80-9b90-11eb-9bf9-15add91f9d09.png)
\[출처: http://research.sualab.com/introduction/2017/11/29/image-recognition-overview-2.html \]


`바운딩 박스(Bounding box)`는 이미지 내에서 물체의 위치를 **사각형으로 감싼 형태의 도형으로 정의**하고 이를 **꼭지점의 좌표로 표현하는 방식**이다.


좌표를 표현하는 방식은 두 가지가 있다. 

#### 절대 좌표 정의
첫 번째는 전체 이미지의 좌측 상단을 원점으로 정의하고 바운딩 박스의 좌상단 좌표와 우하단 좌표 두 가지 좌표로 표현하는 방식이다. 

#### 상대 좌표 정의
두 번째는 이미지 내의 절대 좌표로 정의하지 않고 바운딩 박스의 폭과 높이로 정의하는 방식이다. 

### 3) Intersection over Union
---

![image](https://user-images.githubusercontent.com/70703320/114340184-f51fd780-9b91-11eb-9696-5eb3687ee13a.png)

\[출처: http://research.sualab.com/introduction/2017/11/29/image-recognition-overview-2.html \]

면적의 절대적인 값에 영향을 받지 않도록 두 개 박스의 차이를 상대적으로 평가하기 위한 방법 중 하나로 **IoU(Intersection over Union)** 가 존재한다. 교차하는 영역을 합친 영역으로 나눈 값을 말한다.

위의 그림에서 빨간색 영역 $A_p$는 예측(prediction)과 정답 $A_{gt}$(ground truth)의 교집합인 영역이고 회색 영역이 합집합인 영역일 때, IoU는 빨간 영역을 회색 영역으로 나눠준 값이다.



## Localization을 위한 Target Label

### 1) Target Label
---

localization을 위해 박스의 위치를 표현할 output 노드 4개를 convolution layer로 구성된 백본 네트워크 다음에 추가한다.

![image](https://user-images.githubusercontent.com/70703320/114340466-9575fc00-9b92-11eb-9194-c3b982e68805.png)
[출처: https://youtu.be/GSwYGkTfOKk]

- $p_c$는 물체가 있을 확률
- 물체가 있을 때 $c_1, c_2, c_3$는 각각 클래스 1, 2, 3에 속할 확률 
- $b_x, b_y$는 좌측상단의 점을 표현하는 x축과 y축의 좌표
- $b_h$,$b_w$는 바운딩 박스의 높이와 폭

단, $b_x, b_y, b_h, b_w$는 모두 입력 이미지의 너비 w, 높이 h로 각각 Normalize된 상대적인 좌표와 높이/폭으로 표시된다.


## Detection :: 슬라이딩 윈도 vs 컨볼루션

어떻게 물체를 찾아낼(detection) 수 있을까? 슬라이딩 원도우와 컨볼루션을 통해 알 수 있다. 


### 1) 슬라이딩 윈도우(Sliding Window)
---

전체 이미지를 적당한 크기의 영역으로 나눈 후에, 각각의 영역에 대해 이전 스텝에서 만든 Localization network를 반복 적용해 보는 방식이다.

Localization network의 입력으로 만들기 위해서 원본 이미지에서 잘라내는 크기를 윈도우 크기로 하여, 동일한 윈도우 사이즈의 영역을 이동시키면서(sliding) 수행해주는 방식


### 2) 컨볼루션(Convolution)
---
![image](https://user-images.githubusercontent.com/70703320/114345750-3d90c280-9b9d-11eb-9530-bb869d32d87b.png)
\[출처: https://medium.com/datadriveninvestor/evolution-of-object-recognition-algorithms-i-5803c7be0691 \]

위 그림에서 14X14 크기의 입력에 대해 convolution을 수행했을 때 최종적으로 얻어지는 1X1 사이즈의 출력을 sliding window 영역의 localization 결과라고 본다면, 거꾸로 14X14 크기의 receptive field가 바로 sliding window 영역이 되는 것이다. 

하지만 컨볼루션은 순차적으로 연산이 실행되는 게 아니라 **병렬적으로 동시에 진행되므로 convolution은 속도면에서 훨씬 효율적**이다.

결과적으로 Sliding window를 한번에 Convolution으로 연산을 할 수 있다. 이전 스텝에서 localization을 위한 output이 flat vector 형태로 생성되었다면, convolution의 결과로 얻어지는 1x1 형태의 output도 동일하게 localization 결과로 대체될 수 있을 것이다.


위의 그림에서 입력 이미지가 들어왔을 때 특성을 추출하기 위한 첫번째 스테이지를 거친 후 classifier에서 14x14, 16x16 크기의 입력 두 가지를 볼 수 있다.

`14x14`에서는 1칸에 대한 결과를 볼 수 있고 `16x16`에서는 4칸에 대한 결과를 볼 수 있다.

## Detection :: 앵커 박스, NMS

겹치는 문제들에 해결을 위한 방법 앵커박스와 NMS

### 3) 앵커 박스(Anchor box)
---
![image](https://user-images.githubusercontent.com/70703320/114357528-cbc07500-9bac-11eb-9438-354ff90794e8.png)
\[출처: https://unsplash.com/photos/I4pHtQX23k0/ \]

**앵커 박스(anchor box)** 는 서로 다른 형태의 물체와 겹친 경우에 대응할 수 있다. 일반적으로 차는 좌우로 넓고 사람은 위아래로 길쭉하기 때문에 사람의 형태와 유사한 형태와 차와 유사한 형태의 가상의 박스 두 개를 정의한다.

그림을 통해 살펴보자.

![image](https://user-images.githubusercontent.com/70703320/114357764-19d57880-9bad-11eb-8ea4-d006171ff981.png)
\[출처: http://datahacker.rs/deep-learning-anchor-boxes/ \]

`Anchor box #1`은 사람을 위해 설정한 크기이고 `Anchor box #2`는 차를 위해 설정한 크기이다. $y$의 라벨을 보면 앵커 박스가 2개가 됨에 따라서 output dimension이 두 배가 된 것이고 각각이 각각의 물체를 담당한다.


한 그리드 셀에서 앵커 박스에 대한 물체 할당은 위에서 배운 IoU로 할 수 있다. 인식 범위 내에 물체가 있고 두 개의 앵커 박스가 있는 경우 IoU가 더 높은 앵커 박스에 물체를 할당하게 된다.


#### 바운딩 박스와 앵커 박스의 차이 잡고 가기

> - 바운딩 박스 : 네트워크가 predict한 object의 위치가 표현된 박스로서, 네트워크의 출력이다.
> - 앵커 박스 : 네트워크가 detect해야 할 object의 shape에 대한 가정(assumption)으로서, 네트워크의 입력이다.


### 4) NMS(Non-Max Suppression)
---
![image](https://user-images.githubusercontent.com/70703320/114358318-c0217e00-9bad-11eb-8830-9c77b9fcd413.png)
\[출처: https://www.quora.com/How-does-non-maximum-suppression-work-in-object-detection \]

https://www.youtube.com/watch?v=VAo84c1hQX8

위 그림과 같이 Anchor box를 사용하지 않더라도 2x2격자에 모두 걸친 물체가 있는 경우 하나의 물체에 대해 4개의 Bounding box를 얻게 될 수 있다. 이렇게 겹친 여러 개의 박스를 하나로 줄여줄 수 있는 방법 중 하나가 **NMS(non-max suppression)** 이다.

NMS는 겹친 박스들이 있을 경우 가장 확률이 높은 박스를 기준으로 기준이 되는 IoU 이상인 것들을 없앤다. 이때 IoU를 기준으로 없애는 이유는 어느 정도 겹치더라도 다른 물체가 있는 경우가 존재할 수 있기 때문이다. 


##  Detection Architecture :: two-stage detector vs one-stage detector


딥러닝 기반의 Object Detection 모델은 크게 **Single stage detector**와 **Two stage detector**로 구분할 수 있다. 


![image](https://user-images.githubusercontent.com/70703320/114361325-f8768b80-9bb0-11eb-9433-e41e286e1b59.png)
\[출처: https://medium.com/@jitender_phogat/1-2-introducing-retinanet-and-focal-loss-for-dense-object-detection-7ef9c4901b61 \]

many stage
- 물체가 있을 법한 위치의 **후보(proposals)** 들을 뽑아내는 단계, 이후 실제로 물체가 있는지를 **Classification**과 정확한 바운딩 박스를 구하는 **Regression**을 수행하는 단계가 분리되어 있다. ex) Faster-RCNN

one stage
- 객체의 검출과 분류, 그리고 바운딩 박스 regression을 한 번에 하는 방법이다.



### 1) Two-Stage Detector

#### R-CNN
![image](https://user-images.githubusercontent.com/70703320/114361673-5905c880-9bb1-11eb-8d7b-37a7866e9920.png)
\[출처: https://arxiv.org/pdf/1311.2524.pdf \]

R-CNN은 **물체가 있을 법한 후보 영역을 뽑아내는 "Region proposal" 알고리즘과 후보 영역을 분류하는 CNN**을 사용한다.

이때 Proposal을 만들어내는 데에는 Selective search라는 비신경망알고리즘이 사용되며 후보 영역의 Classification과 바운딩 박스의 regression을 위해 신경망을 사용한다.

region proposal을 selective search로 수행한 뒤 약 2,000개에 달하는 후보 이미지 각각에 대해서 컨볼루션 연산을 수행하게 된다. 이 경우 한 이미지에서 특성을 반복해서 추출하기 때문에 비효율적이고 느리다.


#### Fast R-CNN
![image](https://user-images.githubusercontent.com/70703320/114361786-894d6700-9bb1-11eb-831b-9505c79eb310.png)
\[출처: https://arxiv.org/pdf/1504.08083.pdf \]

Fast R-CNN에서는 **후보 영역의 classification과 바운딩 박스 regression을 위한 특성을 한 번에 추출** 하여 사용한다. 

#### 비교
R-CNN과 같이 이미지를 Sliding Window 방식으로 잘라내는 것이 아니라 해당 부분을 CNN을 거친 특성 맵(Feature Map)에 투영해, 특성 맵을 잘라낸다. 이렇게 되면 이미지를 잘라 개별로 CNN을 연산하던 R-CNN과는 달리 한 번의 CNN을 거쳐 그 결과물을 재활용할 수 있으므로 연산 수를 줄일 수 있다.

이때 잘라낸 특성 맵의 영역은 여러 가지 모양과 크기를 가지므로, 해당 영역이 어떤 클래스에 속하는지 분류하기 위해 사용하는 fully-connected layer에 배치(batch) 입력값을 사용하려면 영역의 모양과 크기를 맞추어 주어야 하는 문제가 생깁니다. 논문에서는 **RoI(Region of Interest) pooling**이라는 방법을 제안해서 후보 영역에 해당하는 특성을 원하는 크기가 되도록 pooling하여 사용합니다.

![image](https://user-images.githubusercontent.com/70703320/114361893-af730700-9bb1-11eb-9733-d5e4e0289b2b.png)

\[출처: https://jamiekang.github.io/2017/05/28/faster-r-cnn/ \]


**Faster R-CNN**
![image](https://user-images.githubusercontent.com/70703320/114362013-d16c8980-9bb1-11eb-8c23-0b2111a877ce.png)
\[출처: https://arxiv.org/pdf/1506.01497.pdf \]

Fast R-CNN은 반복되는 CNN 연산을 크게 줄여냈지만 **region proposal 알고리즘이 병목**이 된다. Faster R-CNN에서는 기존의 Fast R-CNN을 더 빠르게 만들기 위해서 region proposal 과정에서 **RPN(Region Proposal Network)** 라고 불리는 신경망 네트워크를 사용한다.

먼저 이미지에 CNN을 적용해 특성을 뽑아내면, 특성 맵만을 보고 물체가 있는지 알아낼 수 있다. 이때 특성맵을 보고 후보 영역들을 얻어내는 네트워크가 RPN이다.



### 2) One-Stage Detector
---
#### YOLO (You Only Look Once)

![image](https://user-images.githubusercontent.com/70703320/114362267-1b556f80-9bb2-11eb-9752-aaff1c3edb0b.png)
\[출처: https://arxiv.org/pdf/1506.02640.pdf \]

**YOLO**는 이미지를 그리드로 나누고, 슬라이딩 윈도 기법을 컨볼루션 연산으로 대체해 Fully Convolutional Network 연산을 통해 그리드 셀 별로 바운딩 박스를 얻어낸 뒤 바운딩 박스들에 대해 NMS를 한 방식이다. 

논문을 보면 이미지를 7x7짜리 그리드로 구분하고 각 그리드 셀마다 박스를 두 개 regression하고 클래스를 구분하게 한다. 이 방식의 경우 그리드 셀마다 클래스를 구분하는 방식이기 때문에 두 가지 클래스가 한 셀에 나타나는 경우 정확하게 동작하지는 않는다. 하지만 매우 빠른 인식 속도를 가진다.


#### SSD (Single-Shot Multibox Detector)

![image](https://user-images.githubusercontent.com/70703320/114362585-75563500-9bb2-11eb-96d4-f1eebbf4e965.png)
\[출처: https://arxiv.org/pdf/1512.02325.pdf \]

YOLO의 경우 이 특성이 담고 있는 정보가 동일한 크기의 넓은 영역을 커버하기 때문에 작은 물체를 잡기에 적합하지 않는 단점을 해결하고 다양한 크기의 특성 맵을 활용하고자 한 것이 **SSD**이다. SSD는 위의 모델 아키텍쳐에서 볼 수 있듯이 다양한 크기의 특성 맵으로부터 classification과 바운딩 박스 regression을 수행한다. 이를 통해서 다양한 크기의 물체에 대응할 수 있는 detection 네트워크를 만들 수 있다.

## Anchor :: matching bounding box regression

### 1) Matching
---

YOLO와 Faster-RCNN에서 Anchor를 기반으로 Loss를 계산하는 방식에는 두가지 Intersection over Union(IoU)를 threshold로 사용한다. 하나는 **Background IoU threshold** 그리고 다른 하나는 **Foreground IoU threshold**이다. 

Faster RCNN에서  **객체와의 IoU가 0.7이상일 경우 Foreground**로 할당하고 **0.3 이하일 경우는 배경(Background)으로 할당**한다. 0.3과 0.7 중간인 Anchor들은 불분명한 영역으로 학습에 활용하지 않는다.


![image](https://user-images.githubusercontent.com/70703320/114365580-90767400-9bb5-11eb-9173-aa32a15bfd12.png)
\[출처: https://www.pexels.com/photo/grey-sport-utility-vehicle-1687147/ \]


위의 차량을 Detection하기 위한 Detection model을 상상해보면 Feature map의 Grid 한칸마다 다양한 크기 및 Apsect ratio를 가진 Anchor를 만들고 이와 차량의 크기를 비교해서 학습을 해야합니다.

위의 예시를 보면 Anchor의 크기가 차량의 박스 크기에 비해 매우 작은 것을 알 수 있죠. 이 경우 차량 영역에 들어간 Anchor box이라도 교차(intersection)하는 면적이 작기 때문에 IoU가 작아 매칭이 되지 않는 것을 알 수 있습니다. 이러한 이유때문에 우리가 탐지하고자 하는 물체에 따라서 Anchor box의 크기나 aspect ratio를 조정해주어야 합니다.

만약 세로로 긴 물체를 주로 탐지해야 하면 세로로 긴 Anchor box를 많이 만들고 Matching되는 box를 늘려야 학습이 잘될 수 있습니다.


![image](https://user-images.githubusercontent.com/70703320/114365715-af750600-9bb5-11eb-91d3-c5469e86ad91.png)

차량을 Detection하기위해서 Anchor의 크기를 키우고 Aspect raito를 조정하여 가로로 길게 만들 수 있다.

Anchor box와 녹색의 차량 bounding box 간의 IoU를 계산해서 물체와의 IoU가 0.3 이하인 경우는 배경으로 그리고 0.7 이상인 경우는 차량으로 할당되고 두가지 모두에 해당하지 않는 경우에는 Loss가 학습에 반영되지 않도록 한다.

노란색의 경우에는 배경이므로 물체의 bounding box를 추정하기 위한 Regression은 loss에 반영되지 않는다. 따라서 파란색 anchor box만이 regression을 학습할 수 있는 sample이 된다.


### 2) Bounding box Regression
---

![image](https://user-images.githubusercontent.com/70703320/114365843-d29fb580-9bb5-11eb-971f-d4be16aa851a.png)

Anchor box는 물체를 예측하기 위한 기준이되는 Box이다. Anchor box에서 Bounding box는 어떤 식으로 계산이 되는지 살펴보자.

#### YOLOv3를 기준
YOLOv3의 전신인 YOLOv2부터 Bounding box Regression 방식을 사용하게 된다.

이때, 각 Anchor box에 대응되는 network는 $t_x, t_y, t_w, t_h$ 의 4가지 output으로 bounding box를 regression해서 정확한 box를 표현하게 된다.


Bounding box를 예측하기 위해 bounding box의 중심점($b_x, b_y$), 그리고 width($b_w$)와 height($b_h$)을 찾아야 한다.

위 그림에 있는 수식과 같이 anchor box의 정보 $c_x, c_y, p_w, p_h$ 와 연관지어 찾는 방법을 사용한다.

기존의 Anchor box 위 좌측 상단이 $c_x, c_y$ 이고 width, height가 $p_w, p_h$ 이다. 이를 얼마나 x축 또는 y축방향으로 옮길지 그리고 크기를 얼마나 조절해야하는지를 예측하여 물체의 bounding box를 추론하게 된다.